In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_stats,get_bureau_feats_2
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy

In [2]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [3]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle(DATA_DIR+'bureau_data.pkl')

In [4]:
train.shape,test.shape,bur_df.shape

((128655, 26), (14745, 25), (591000, 32))

In [5]:
train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
test['MaturityDAte'] = pd.to_datetime(test['MaturityDAte'])


bur_df['DATE-REPORTED'] = pd.to_datetime(bur_df['DATE-REPORTED'])
bur_df['dpd_strin_var'] = bur_df['dpd_string'].fillna('').apply(lambda x: [x[y-3:y] for y in range(3, len(x)+3, 3)] if x.find('E')==-1 else ['000'])

In [6]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [7]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10 00:00:00,4022465,1568.0,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31 00:00:00,4681175,1062.0,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17 00:00:00,25328146,1060.0,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29 00:00:00,13021591,1060.0,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08 00:00:00,3291320,1046.0,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


#### create features for the period of LTFS loan present in the application data, like time to next loan, time to last loan,CIBIL fc on the tradelines originated in the period

In [8]:
bur_df = bur_df.merge(df[['ID','DisbursalDate','MaturityDAte','DisbursalAmount']],on='ID')
bur_df.sort_values(['ID','DISBURSED-DT'],inplace=True)

In [9]:
(bur_df['SELF-INDICATOR']==True).value_counts(1)

False    0.694228
True     0.305772
Name: SELF-INDICATOR, dtype: float64

In [10]:
bur_df.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,min_reported_date,max_reported_date,dpd_str_len,dpd_string,correctedOVERDUE-AMT,correctedDISBURSED-AMT/HIGH CREDIT,correctedCURRENT-BAL,dpd_strin_var,DisbursalDate,MaturityDAte,DisbursalAmount
8,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2013-03-31,2016-02-29,108,0000000000000000000000000000000000000000000000...,0.0,275000.0,0.0,"[000, 000, 000, 000, 000, 000, 000, 000, 000, ...",2012-02-10,2016-01-15,275000.0
6,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-01-30,2017-09-21,NaT,NaN,"5,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0000000000000870540270270000320000000000...,"0,,0,0,0,0,0,0,0,13060,13060,25982,38767,51417...","0,,0,0,0,0,3064,3064,3064,2972,0,2802,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-10-31,2017-09-30,108,000DDD0000000000000870540270270000320000000000...,0.0,500000.0,0.0,"[000, DDD, 000, 000, 000, 000, 087, 054, 027, ...",2012-02-10,2016-01-15,275000.0
7,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-07-31,Individual,Closed,2013-06-11,2017-07-01,NaT,NaN,"4,00,000",NaN,0,NaN,0,0.0,NaN,"20170731,20170601,20170531,20170430,20170331,2...",000DDD0220210220190220220000270000000000000000...,"0,,0,10487,20862,31126,41281,51328,51328,71101...","0,,1722,1722,1722,1722,1722,1722,0,1625,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-08-31,2017-07-31,108,000DDD0220210220190220220000270000000000000000...,0.0,400000.0,0.0,"[000, DDD, 022, 021, 022, 019, 022, 022, 000, ...",2012-02-10,2016-01-15,275000.0
5,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-10-31,2017-09-30,108,000DDD0270260270260270240270270000320000000000...,0.0,300000.0,0.0,"[000, DDD, 027, 026, 027, 026, 027, 024, 027, ...",2012-02-10,2016-01-15,275000.0
3,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaT,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,2018-03-31,2018-04-30,6,030000,37873.0,37352.0,37873.0,"[030, 000]",2012-02-10,2016-01-15,275000.0


In [11]:
bur_df['ACCT-TYPE'].value_counts(1).head()

Tractor Loan                                  0.349277
Gold Loan                                     0.163113
Business Loan Priority Sector  Agriculture    0.138684
Kisan Credit Card                             0.061856
Auto Loan (Personal)                          0.054327
Name: ACCT-TYPE, dtype: float64

In [15]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>temp_df.DisbursalDate)&(temp_df['ACCT-TYPE']=='Gold Loan')
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats_gold = pd.DataFrame(data_outputs)    

16731 (57822, 36)
CPU times: user 7.93 s, sys: 374 ms, total: 8.31 s
Wall time: 35.8 s


In [16]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>temp_df.DisbursalDate)&(temp_df['ACCT-TYPE']=='Tractor Loan')
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats_trac = pd.DataFrame(data_outputs)    

36139 (52926, 36)
CPU times: user 18.5 s, sys: 709 ms, total: 19.2 s
Wall time: 1min 17s


In [17]:
bur_feats_gold.columns = ['gold_'+x if x!='ID' else x for x in bur_feats_gold.columns]
bur_feats_trac.columns = ['trac_'+x if x!='ID' else x for x in bur_feats_trac.columns]

In [18]:
df[['ID']].merge(bur_feats_gold,on='ID',how='left').merge(bur_feats_trac,on='ID',how='left').fillna(0).to_pickle('gold_trac_future_from_disb_date_feats.pkl')

### features for the accounts within 2 years of disrbursal of current loan

In [11]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*2)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats1 = pd.DataFrame(data_outputs)    

47191 (95480, 36)
CPU times: user 22.5 s, sys: 880 ms, total: 23.3 s
Wall time: 1min 41s


In [12]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>(temp_df.DisbursalDate+dt.timedelta(days=365*2)))&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*4)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats2 = pd.DataFrame(data_outputs)    

43875 (89145, 36)
CPU times: user 21.3 s, sys: 910 ms, total: 22.2 s
Wall time: 1min 35s


In [13]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>(temp_df.DisbursalDate+dt.timedelta(days=365*4)))&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*10)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats3 = pd.DataFrame(data_outputs)    

32304 (88485, 36)
CPU times: user 16.3 s, sys: 959 ms, total: 17.2 s
Wall time: 1min 10s


In [14]:
bur_feats1.shape,bur_feats2.shape,bur_feats3.shape

((47191, 83), (43875, 83), (32304, 83))

In [15]:
bur_feats1.columns = ['2_yr_frm_disb'+x if x!='ID' else x for x in bur_feats1.columns]
bur_feats2.columns = ['2-4_yr_frm_disb'+x if x!='ID' else x for x in bur_feats2.columns]
bur_feats3.columns = ['grtr_4_yr_frm_disb'+x if x!='ID' else x for x in bur_feats3.columns]

In [16]:
df[['ID']].merge(bur_feats1,on='ID',how='left').merge(bur_feats2,on='ID',how='left').merge(bur_feats3,on='ID',how='left').fillna(0).to_pickle('future_from_disb_date_feats.pkl')